# Introduction

For the 9th Workshop in Biomedical Engineering (http://wbme.fc.ul.pt/) a specific workshop was prepared to showcase how some of IBM Watson's API could be used; given the target audience and the overall themes of the WBME I had the following 

* Use one or more Watson APIs
* Build a use-case which makes sense within the context of the WBME
* Make the workshop interactive but also useful for those just watching
* Have as little software requirements as possible
* Make it useful even after the workshop, which includes being able to distribute it in some way

After some days experimenting with different options I settled on:

* IBM Watson Visual Recognition API
* A dermoscopic database of images that show skin lesions, both benign and malignate: the PH(2) database from the ADDI project (https://www.fc.up.pt/addi/ph2%20database.html).
* Use IBM Data Science Experience to create, share and enable all to work on their one copies with no software requirements by making use of the Jupyter notebook support with enhanced user interface.

The initial developed was done using the `Rmd` format and it only required slight changes; one of the changes required the use of a specific zip file for the PH(2) dataset since the original one was in a RAR file and this compression mechanism isn't supported directly in R, which means that I was decompressing it using a system command, not something feasible when using a Spark execution environment. That was the _only_ change to the original PH(2) archive (which means that there is not actual change in the contents).

I would like to thank the 8th WBME organisers for the opportunity and in particular to all of those who attended the workshop and that actively participated in it; a particular thank you to Sara Lobo (Biomedical Engineering, FCT/NOVA) in which laptop we ended up working together for the final part of the workshop that ended (successfully!) on the Student's Room of the Faculty of Sciences of the University of Lisbon more in the style of a group debugging session than a formal workshop - much to the advantage of all involved.


## Concerning the topic

The use if visual recognition for detection of malignant skin lesions is something that has been studied intensively and has been the topic of many specialised papers and studies. While I've chose this topic due to being aligned with a bio-medical engineering event the goal of the Watson API workshop is to explain how to use the Visual Recognition API, not to build a useful model for something with such implications as melanoma detection.

I've added some relevant bibliography that clearly show the complexity and the possible approaches which are possible, including image pre-processing, the importance of masking, different strategies in terms of classification and many more, for those who want to explore the matter in more depth.

# Setting up the environment

Throughout this paper we will need to use and install some `R` packages, and given the interactive nature of the notebook it is better to add all of those initial requirements in a single code statement, that way we can run it once and be done with it.

In [ ]:
## Load the `caret` library, which includes the partitioning function we will use
library(caret)
## Package e1071 is used by `caret` for the confusion matrix
install.packages('e1071', dependencies=TRUE)
# Install needed packages for image conversion
install.packages("bmp")
install.packages("jpeg")
install.packages("pixmap")
## Load the image-related libraries
library(bmp)
library(jpeg)
library(pixmap)
library(jpeg)
library(grid)
## For the REST API
library(jsonlite)
library(httr)

An IBM Bluemix (https://www.ibm.com/cloud-computing/bluemix/) account is needed; during the workshop this was the first step done but the process is simple: sign up and create a free account and get access to the entire catalogue, including Watson APIs.

# Obtaining and preparing the data

The first step we are going to take is downloading the PH(2) database and preparing the data; these step is fundamental in terms of Data Science and we will spend some time going through it step-by-step to make sure that we have all the needed actions properly automated.

The focus on having everything done "in code" is important since it enables repeatability, which is crucial.

## Cleaning up

This seems strange: why clean up when we are just starting? Given that we are in an interactive (and iteractive) environment which keeps data between runs it is assumed that we will want to run the code several times, and this first step is to remove all the files which we will create during this document. If there is nothing created yet it's not a problem: the `unlink` function deletes what exists and if it doesn't it does nothing.

There is a sole exception: we will not delete de compressed archive since it doesn't change and this avoids having to download the same context several times.


In [ ]:
## Delete previous runs
zip_dir <-"ph2-training-files"
## If present remove existing archives from previous runs
unlink(zip_dir, recursive=T,force=T)
unlink("training-set-positive.zip", force=T)
unlink("training-set-negative.zip", force=T)
unlink("parameters.json", force=T)
unlink("*.csv", force=T)
unlink("PH2Dataset", force=T, recursive=T)
## Uncomment to delete the database zip as well
##unlink("PH2Dataset.zip", force=T)

## Getting the dataset

The first action is to download the PH(2) database, available as a downloadable compressed archive; we set up some variables - the URL where the file is and the name of the zip file - and check for the zip file locally, bypassing the download if it already existing.

In [ ]:
## New dataset
ph2_url  <- "https://www.dropbox.com/s/9a962jfcrs5x4iq/PH2Dataset.zip?dl=0"
ph2_archive <- "PH2Dataset.zip"

### Check for zipped dataset file: if it doesn't exist then download it
if (!file.exists(ph2_archive)) {
    print("Downloading PH(2) archive")
    download.file(ph2_url, destfile=ph2_archive, method="wget")
} else {
    print("File already exists, skipping download")
}


We should have the PH(2) zip zile (and just that file) in our working directory.

In [ ]:
dir()

Now that we have the archive we unzip it.

In [ ]:
dir(".")
unzip(ph2_archive)

## Initial cleanup

The structure of the newly created directory

In [ ]:
dir("PH2Dataset")

Of particular interest to us is the `PH2_dataset.txt` file that contains a text-based description of the obervations. Let's read the file into a variable and examine some of the content.

In [ ]:
ph2_txt <- readLines("PH2Dataset/PH2_dataset.txt")
head(ph2_txt)

It's a file best read using a monospaced font but one can clearly see that it has a header and then the observations, which columns divided by vertical bars; this is something easy for us to understand but not yet ideal to be used programatically, so we will do some transformations to convert it into something that uses a comma to separate the differente fields - a CSV file which can be directly imported into R, and which we will write to disk (something optional since we could just use the result of the transformations directly).

In [ ]:
## Convert the txt to a csv file; due to the format this requires
## several operations.
ph2_new <- gsub ("^\\|\\|", "", fixed=FALSE,ph2_txt[1:201]) # Delete the || in the beginning of the file 
ph2_new <- gsub ("\\|\\|$", "", fixed=FALSE,ph2_new)        # Delete the || at the end of the file
ph2_new <- gsub ("||", ",", fixed=T,ph2_new)                # Replace all the remaining || with a comma
ph2_new <- gsub ("|", ",", fixed=T,ph2_new)                 # Replace all the | with a comma

## Save the result to a file
writeLines(ph2_new, "ph2_dataset.csv")
## ... and read that file into a variable
ph2_table <- read.csv("ph2_dataset.csv", header=T)

We now have a CSV file in our working directory,

In [ ]:
dir()

... and a `ph2_table` variable which is an R data frame resulting from the import of the CSV file.

In [ ]:
str(ph2_table)
head(ph2_table)

## Creating the training and testing sets

Some columns are interpreted as numeric although they are factors; additionally we remove the "atypical nevus" images since for our purposes a smaller dataset is desirable since we want to keep the training time of the model small enough and also avoid reaching the daily allowance of images; this has the effect of removing a group of images which are ambiguous and will certainly result in a model which is simples in the sense that it works has been trained with less corner cases.

In [ ]:
ph2_table$Clinical.Diagnosis <- as.factor(ph2_table$Clinical.Diagnosis)
ph2_table$Asymmetry <- as.factor(ph2_table$Asymmetry)
ph2 <- ph2_table[!ph2_table$Clinical.Diagnosis == "1", ]
ph2$Clinical.Diagnosis <- droplevels(ph2$Clinical.Diagnosis)

In [ ]:
## We only have two factors now
print(ph2$Clinical.Diagnosis)

We now split the training data into two different sets, training (70%) and testing (30%), using the Clinical Diagnosis as the outcome

In [ ]:
inTrain <- createDataPartition(y=ph2$Clinical.Diagnosis, p=0.7, list=FALSE)
training_set <- ph2[inTrain, ]
testing_set <-  ph2[-inTrain, ]

In [ ]:
## See what we have so far
head(training_set)
head(testing_set)

Our approach is straightforward and does not consider the need for
oversampling/bootstraping: we will use the same proportion present in
the pruned dataset.

In [ ]:
print(sprintf("Common Nevus: %s%%", round(nrow(ph2[ph2$Clinical.Diagnosis == "0", ]) / nrow(ph2) *100, 0)))
print(sprintf("Melanoma: %s%%",     round(nrow(ph2[ph2$Clinical.Diagnosis == "2", ]) / nrow(ph2) *100, 0)))
## Plot
barplot(table(ph2$Clinical.Diagnosis))

## Positive and negative

To use IBM Watson's Visual Recognition we need both positive and
negative sets to test the network; we will use our training set and
divide it into a positive set which includes the melanoma diagnosis
and a negative one which includes the rest.


In [ ]:
## Total number of cases
sprintf("Total number of cases in training set: %d",nrow(training_set))
## Benign and malignal cases in the training set

sprintf("Total number of bening cases in training set: %d", nrow(training_set[training_set$Clinical.Diagnosis == "0",]))
sprintf("Total number of malignant cases in training set: %d", nrow(training_set[training_set$Clinical.Diagnosis == "2",]))

## Calculate the percentages...
print(sprintf("Common Nevus: %s%%", round(nrow(training_set[training_set$Clinical.Diagnosis == "0", ]) / nrow(training_set) *100, 0)))
print(sprintf("Melanoma: %s%%",     round(nrow(training_set[training_set$Clinical.Diagnosis == "2", ]) / nrow(training_set) *100, 0)))

## ... and plot them
barplot(table(training_set$Clinical.Diagnosis))

## Create variables for both sets to make it clearer
training_positive <- (training_set[training_set$Clinical.Diagnosis == "2",])
training_negative <- (training_set[training_set$Clinical.Diagnosis == "0",])


## Files and paths

The Visual Recognition API accepts the images as a zip file so we use
the image names to build the adequate archives, but before let's explore the database through the directory structure in a bit more detail.

As we saw we have at the toplevel some files and a directory:

In [ ]:
dir("PH2Dataset")

Inside the images directory there are other directories, one for each file. We will use an option to `dir` to obtain the full path of the files and directories to make it clearer.

In [ ]:
dir("PH2Dataset/PH2 Dataset images", full.names=T)

Each of these directories has a similar structure, and we will use a random one to list all the contents.

In [ ]:
name_sample <- training_set[sample.int(nrow(training_set),1), ]
name_sample$Name

To get there we just need to build the file path - this will be very important in the next step - by simply appending the file name to the images directory.

In [ ]:
dir(sprintf("PH2Dataset/PH2 Dataset images/%s", name_sample$Name), full.names=T, recursive=T)

Is that directory empty? We actually know it isn't (you can download the archive and check it yourself), so what happened? The answer lies in the `Name` field: if we check it a bit better we can see that it contains extra characters.

In [ ]:
sprintf("PH2Dataset/PH2 Dataset images/%s", name_sample$Name)

Since we didn't clean the `Name` column properly it contains extra whitespace, which is passed into `dir`. The solution is to trim all the names, removing any whitespace.

Note that this, as many other actions we will perform, would be better done to the initial `ph2_table` data frame that contains the complete number of observations and from which we would then extract the needed datasets, but since we're learning as we go we will correct the derived datasets seperately.

In [ ]:
## Trim function: remove unneeded whitespace
trim <- function (x) gsub("^\\s+|\\s+$", "", x)
training_set$Name <- trim(training_set$Name)
testing_set$Name <- trim(testing_set$Name)

With this fixed let's see how it works.

In [ ]:
## Get a new sample
name_sample <- training_set[sample.int(nrow(training_set),1), ]
## Check the file path
sprintf("PH2Dataset/PH2 Dataset images/%s", name_sample$Name)

Looks correct so now we can try to obtain the directory listing of the image directory again, and hopefuly it will work now.

In [ ]:
dir(sprintf("PH2Dataset/PH2 Dataset images/%s", name_sample$Name), full.names=T, recursive=T)

Great, it worked.

The image we are going to use if the one in the _Dermoscopy Image_ directory, and we will not use any of the two (one always present, the other optional) additional images made available which are _masks_ which identify the lesion itself from the surrounding tissue. This is explained in the `Readme.txt` file, which should actually be the very first thing we should read in order to understand the database so let's take a look now.

In [ ]:
readLines("PH2Dataset/Readme.txt")


We are now able to add a correct file image path to each observation and so we add a new column - `File` - that contains that information.

In [ ]:
## Trim the entries in the positive and negative sets
training_positive$Name <- trim(training_positive$Name)
training_negative$Name <- trim(training_negative$Name)
## Add the file image path to each observation
training_positive$File <- sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.bmp", training_positive$Name, training_positive$Name, training_positive$Name)
training_negative$File <- sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.bmp", training_negative$Name, training_negative$Name, training_negative$Name)
## Since we could forget about this latter we will do the same to the testing set right now
testing_set$File <- sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.bmp", testing_set$Name, testing_set$Name, testing_set$Name)

We can now see that we have the file path in an additional column

In [ ]:
head(training_positive)
head(training_negative)
head(testing_set)

## Image format conversion

Everything looks fine... but there's nothing like making sure things are actually working so we will try to load and display an image from two random samples.

In [ ]:
## Pick a random sample from the training set
negative_sample <- training_negative[sample.int(nrow(training_negative),1), ]
positive_sample <- training_positive[sample.int(nrow(training_positive),1), ]

## Use read.bmp to read the image and then create a pixmapRGB object that can be "plotted"
negative_image <- pixmapRGB(read.bmp(negative_sample$File))
positive_image <- pixmapRGB(read.bmp(positive_sample$File))

## We use plot to display the image, and par to display them in a single row, side by side
##par(mfrow=c(1,2))
plot(negative_image, sub = negative_sample$Name)
plot(positive_image, sub = positive_sample$Name)

Now that we have confirmed that the image paths are correct we need to build the zip archives; Watson Visual recognition receives the images as zip files which contain all the images that will be used to build the model.

First we create some auxiliary directories that will be used to make the two zip files.

In [ ]:
zip_dir <-"ph2-training-files"
positive_dir <- file.path(zip_dir,"positive")
negative_dir <- file.path(zip_dir,"negative")

dir.create(zip_dir)
dir.create(positive_dir)
dir.create(negative_dir)
dir(zip_dir)

We just need to copy the images and create the zip right? Actually, there is still one important thing missing: Visual Recognition accepts files in JPEG and PNG format, and the PH(2) dataset has the files as BMP. We need to convert them first, something that is also possible with some additional R libraries.

Each image will be read, converted to JPEG and saved in the same directory as the original, and we will then copy them to specific directories that will be used to build the _positive_ and _negative_ archives. These approach has the advantage of working for the testing set without any change - remember that there is no "positive" or "negative" split in the testing set.

Do note that, as always, there are many different ways to go about this transformation; in the initial version the `magick` library was used and it makes the process quite straightforward and arguably more readable. The downside is that it depends on external libraries that make it less portable, hence the use of the `jpeg` and `bmp` libraries.

In [ ]:
## Pick a new sample
negative_sample <- training_negative[sample.int(nrow(training_negative),1), ]
negative_image <- pixmapRGB(read.bmp(negative_sample$File))
##Write the JPEG
writeJPEG(getChannels(negative_image), target =  sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", negative_sample$Name, negative_sample$Name, negative_sample$Name), quality = 1)

## Read the JPEG
jpeg <- readJPEG(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", negative_sample$Name, negative_sample$Name, negative_sample$Name))

## Display the JPEG
grid.raster(jpeg)

Now that we have a way to convert to JPEG we will convert all the images; this means looping through all observations, and for that we make a function that encapsulates the conversion process.

In [ ]:
## This function converts a given image (provided as the name, without extension) to JPEG and saves it as a side-effect before returning the JPEG image as the return value.
bmp_to_jpeg <- function (name) {
    ## Disable warnings from pixmapRGB
    oldw <- getOption("warn")
    options(warn = -1)

    ## Read the BMP file
    bmp_image <- pixmapRGB(read.bmp(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.bmp", name, name, name)))
    ## Write the JPEG
    writeJPEG(getChannels(bmp_image), target =  sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", name, name, name), quality = 1)
    #print(dir(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image", name, name)))

    ## Read the JPEG and use it as the return value
    #readJPEG(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", name, name, name))
    options(warn = oldw)
}

Now that we can read a BMP file, convert into JPEG and save it we just need to apply these steps to all cases in the datasets

In [ ]:
## Apply the transformation to all rows
for (i in 1:nrow(training_positive))
{
   bmp_to_jpeg(training_positive[i,"Name"])   
}
print("Done")

Now we should have a JPEG image along the original BMP file

In [ ]:
## Pick another sample and check if there
positive_sample <- training_positive[sample.int(nrow(training_positive),1), ]
dir(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/", positive_sample$Name, positive_sample$Name))

## Read the JPEG
jpeg <- readJPEG(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", positive_sample$Name, positive_sample$Name, positive_sample$Name))
## Display the JPEG
grid.raster(jpeg)

Now that we have all the images in the positive set as JPEG we need to do the same to the rest of the images - remember, this would be something that could and should be done to the original `ph2_table` since it would only have to be done once.

In [ ]:
## Do the same for the negative example
for (i in 1:nrow(training_negative))
{
   bmp_to_jpeg(training_negative[i,"Name"])
}
print("Done")

We also check if it worked for this set:

In [ ]:
negative_sample <- training_negative[sample.int(nrow(training_negative),1), ]
dir(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/", negative_sample$Name, negative_sample$Name))

## Read the JPEG
jpeg <- readJPEG(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", negative_sample$Name, negative_sample$Name, negative_sample$Name))
## Display the JPEG
grid.raster(jpeg)

We shouldn't forget about the testing set - we will want to submit images for testing and the same restrictions on file format apply.

In [ ]:
## Do the same for the testing example
for (i in 1:nrow(testing_set))
{
   bmp_to_jpeg(testing_set[i,"Name"])   
}
print("Done")

... and check if it worked.

In [ ]:
testing_sample <- testing_set[sample.int(nrow(testing_set),1), ]
dir(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/", testing_sample$Name, testing_sample$Name))

## Read the JPEG
jpeg <- readJPEG(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", testing_sample$Name, testing_sample$Name, testing_sample$Name))
## Display the JPEG
grid.raster(jpeg)

The `File` field of each observation points to the BMP file; since we want to operate on the JPEG files that we created we simply replace that field (another option woild be to add a new column, thus keeping a reference to both files). 

In [ ]:
## Get fresh samples again
positive_sample <- training_positive[sample.int(nrow(training_positive),1), ]
negative_sample <- training_negative[sample.int(nrow(training_negative),1), ]
testing_sample <- testing_set[sample.int(nrow(testing_set),1), ]

print("Before changing to JPEG")
head(positive_sample)
head(negative_sample)
head(testing_sample)

training_positive$File <- sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", training_positive$Name, training_positive$Name, training_positive$Name)
training_negative$File <- sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", training_negative$Name, training_negative$Name, training_negative$Name)
testing_set$File <- sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", testing_set$Name, testing_set$Name, testing_set$Name)

positive_sample <- training_positive[sample.int(nrow(training_positive),1), ]
negative_sample <- training_negative[sample.int(nrow(training_negative),1), ]
testing_sample <- testing_set[sample.int(nrow(testing_set),1), ]

print("After changing to JPEG")
head(positive_sample)
head(negative_sample)
head(testing_sample)

## Read the JPEG
#jpeg <- readJPEG(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", negative_sample$Name, negative_sample$Name, negative_sample$Name))

## Display the JPEG; we use grid.newpage() since otherwise grid.raster() would only show the last image.
grid.newpage()
grid.raster(readJPEG(positive_sample$File))
grid.newpage()
grid.raster(readJPEG(negative_sample$File))
grid.newpage()
grid.raster(readJPEG(testing_set$File))


## Creating the archives

Let's review some of the steps we've done up until this point:

- Downloaded the PH(2) archive and extracted it
- Imported the description of all observations
- Created the testing and training datasets
- Added the file path to the observations
- Converted all images to JPEG

We're very close to actually starting to use Watson! This preliminary steps are (and I'm stressing this again on purpose) fundamental for any Data Science project: parsing text files and converting images is hardly as attractive as using a Deep Learning framework such as Watson Visual Recognition, but it's a necessary step that when done correctly has a huge positive impact on the ability to repeat and share the experiment.

Watson Visual Recognition trains a model by analysing a zip file with the _positive_ examples, and optionally _negative_ examples. We will use this to train a classifier for melanoma, supplying examples of images which are malignant (the positive archive) and benign (the negative archive).

Remember that we have already created the two directories previously.

In [ ]:
dir(zip_dir, full.names=T)

We can now copy the JPEGs to their respective directory, beginning with the ones form the positive set.

In [ ]:
## Copy the images in the positive set
for (i in 1:nrow(training_positive))
{
    print(sprintf("%s -> %s",training_positive[i,"File"],positive_dir))
    file.copy(training_positive[i,"File"],positive_dir)

}
print("Directory listing:")
dir(positive_dir)

... and now the negative set.

In [ ]:
## Copy the images in the negative set
for (i in 1:nrow(training_negative))
{
    print(sprintf("%s -> %s",training_negative[i,"File"],negative_dir))
    file.copy(training_negative[i,"File"],negative_dir)

}
print("Directory listing:")
dir(negative_dir)

With the images in the appropriate directories it's just a matter of zipping them to create the two archives.

In [ ]:
## Create the archives

## Create the positive archive
WD <- getwd()
setwd(positive_dir)
zip("../../training-set-positive.zip", dir("."), flags="-j9X", zip="/usr/bin/zip")
setwd(WD)

## Create the positive archive
WD <- getwd()
setwd(negative_dir)
zip("../../training-set-negative.zip", dir("."), flags="-j9X", zip="/usr/bin/zip")
setwd(WD)


We now have two zip files which contain the positive and negative sets that will be used to train Watson, composed of JPEG images.

In [ ]:
dir(".")
file.info("training-set-positive.zip")
file.info("training-set-negative.zip")

# Using Watson Visual Recognition

It's (finally!) time to train our model!

## Credentials

We will need the Visual Recognition API key to submit our datasets for training; this information is available in the Bluemix console after adding the Visual Recognition service, something which can be done by logging in Bluemix (https://console.ng.bluemix.net/login), searching for _Visual Recognition_ in the catalogue and adding the service.

This will add the Visual Recognition service to your dashboard (you can accept the default values that are suggested during the creation of the service), and in the details of the service there is a 'Service Credentials' pane which will display the list of credentials and a 'View Credentials' option with something similar to the following JSON string:

```json
{
  "url": "https://gateway-a.watsonplatform.net/visual-recognition/api",
  "note": "It may take up to 5 minutes for this key to become active",
  "api_key": "fcccc0bd44bd59db50121xxxx559e6474892yyy-"
}
```
You will need to copy the `url` and copy it to the `api_end` variable bellow (by replacing the characters within quotes, if needed since the `url` is likely to be the same) and the `api_key` to the variable with the same name.


In [ ]:
## API Endpoint: change to the correct value obtained from the "Credentials" pane in the Visual Recognition service
api_endp  <-  "https://gateway-a.watsonplatform.net/visual-recognition/api"
### API Key: change to the correct value obtained from the "Credentials" pane in the Visual Recognition service
api_key  <- ""

## Creating the model

The creation of a model is straightforward, but as those who were involved in the workshop  know we need to be extra careful to make the code resilient to multiple runs; the original approach was to imediately make a request for Watson Visual Recognition to create a model, and then save the classifier ID that is generally returned by that call.

The problem? If one runs the code a second time (on purpose or by mistake) this will fail since the default free plan of Visual Recognition allows for one classifiers to exist at a time, which means that asking for the creation of the model a second time will _not_ return a classifier ID but an error message. This would trickle down the code and essentially render the code cell inoperative.

The present solution is slightly more complex but mostly because it adds more code, since the approach is quite simple:

1. First, ask for a list of existing classifiers
2. Check the answer: if a classifier ID exists then by default keep it (but offer the option to delete it by checking a specific variable that )
3. If there isn't any classifier then create a new one

This should (hopefuly!) work for all situations and make the code resiliente to multiple runs.

To help us in interacting with the Visual Recognition REST API (cf. https://watson-api-explorer.mybluemix.net/apis/visual-recognition-v3 for the full API reference) we define some functions to retrieve and delete classifiers.

In [ ]:
## Retrieve existing classifiers (if any)
retrieve_classifiers <-  function (api_key, api_endp)
{
    req <- GET(sprintf("%s/v3/classifiers/?api_key=%s&version=2016-05-20", api_endp, api_key))

}

## Delete an existing classifier
delete_classifier <-  function (api_key, api_endp, classifier_id)
{
    req <- DELETE(sprintf("%s/v3/classifiers/%s?api_key=%s&version=2016-05-20", api_endp, classifier_id,api_key))

}


Before proceeding further we must make sure that we have added the corrent information above in the `api_endp` and `api_key`; the following code should return a JSON string
with or without a classifier and associated fields (name, status, etc), but it shouldn't return an error message. The following is an example of the most common error which results from an invalid API key (likely because it wasn't properly copied and pasted):

```json
{
    "status": "ERROR",
    "statusInfo": "invalid-api-key"
}
````


In [ ]:
## Get a list of classifiers to test the main API settings
classifiers_response  <- retrieve_classifiers(api_key, api_endp)
content(classifiers_response,"text")

With that hopefuly out of the way we will now check for an existing classifier; we will be using the `jsonlite` library to convert the JSON strings into R objects.

In [ ]:
## Get the text from the response and parse the JSON into a list
classifiers <- fromJSON(content(classifiers_response,"text"))

## Check the result
classifiers
str(classifiers)

This allows us to more easily work with what the Visual Recognition API answers in terms of the R code we are developing.

In [ ]:
## Check 
if (any(names(classifiers) == "classifiers")) {
        print(sprintf("Existing classifier %s exists, keeping it", classifiers$classifiers$classifier_id))
        classifier_id <- classifiers$classifiers$classifier_id
} else {
    print("No classifiers exist")
         classifier_id <- NULL
}
classifier_id

The following variable determines the behaviour to addopted when a model already existing and should be changed if a different behaviour is desired; the default value, `keep`, will keep the existing model, change it to `new` to delete and retrain.

In [ ]:
## This variable is used to determine what to do when an existing model exists. It can have one of two values:
##   keep: will keep the existing model and use it
##   new: will delete the existing model and train a new one
## 
## It is only checked when an existing model exists.

behaviour <- "keep"

The following code uses

1. Information about existing classifiers
2. The desired behaviour as expressed by the `behaviour` variable

to determine what to do. The code can look daunting but it only does some checking and decides on what to do based on the above two points.

In [ ]:
## Decide on the training action
if(is.null(classifier_id)) {
    ## Classifier is new, train new model
    print("No existing classifier detected, training new model.")
    req  <- POST(sprintf("%s/v3/classifiers?api_key=%s&version=2016-05-20", api_endp, api_key), body = list(melanoma_positive_examples=upload_file("training-set-positive.zip"), negative_examples=upload_file("training-set-negative.zip"), name = "melanoma"), encode = "multipart")
    reply  <- fromJSON(content(req, "text"))
    classifier_id  <-  reply$classifier_id
} else {
    ## Classifier isn't NULL, so we either keep it or retrain it, depending on the variable "behaviour"
    if(behaviour == "keep") {
        ## Keep it
        print(sprintf("Classifier exists and defined behaviour set to keep, reusing classifier %s", classifier_id))
    } else if(behaviour == "new") {
        print("Classifier exists and defined behaviour set to new, deleting classifier and training new model")
        delete_classifier(api_key, api_endp, classifier_id)
        req  <- POST(sprintf("%s/v3/classifiers?api_key=%s&version=2016-05-20", api_endp, api_key), body = list(melanoma_positive_examples=upload_file("training-set-positive.zip"), negative_examples=upload_file("training-set-negative.zip"), name = "melanoma"), encode = "multipart")
        reply  <- fromJSON(content(req, "text"))
        classifier_id  <-  reply$classifier_id
    } else {
        print("Unknown value for variable \"behaviour\", please choose \"keep\" or \"new\"")
    }
    
}

##classifier_id <- 
##reply  <- retrieve_classifiers(api_key, api_endp)
##classifier_id  <-  reply$classifier_id
##retrieve_classifiers(api_key,api_endp)

We now have a classifier id, either because we created the model for the first time or because we are reusing a previous model.

In [ ]:
print(classifier_id)

We can check the status of the classifier with a simple `GET` call

In [ ]:
## The following indicates the status of the classifier: it will return "training" or "ready"
req <- GET(sprintf("%s/v3/classifiers/%s?api_key=%s&version=2016-05-20", api_endp, classifier_id, api_key))
jsonlite::prettify(content(req,"text"))

If it's listed as *ready* we can proceed (this model should take less than 5 minutes to train), so run the following code until it is ready.

In [ ]:
## The "status" can be used to check if the model is ready, such as in this simple example
req <- GET(sprintf("%s/v3/classifiers/%s?api_key=%s&version=2016-05-20", api_endp, classifier_id, api_key))
if (fromJSON(content(req, "text"))$status == "ready")
    {
        print("Classifier ready")
    } else {
        print("Classifier in training, please wait")
}


We have trained a classifier and now it's time to test it.

## Testing the classifier

Now that we have trained our classifier we will use images from our test set to see how Watson classifies them; we define a `classify_image` function to encapsulate the HTTP POST request and make the code simpler to follow.

In [ ]:
## This function uses the Visual Recognition API to submit a file for classification
classify_image  <- function (api_key, api_endp, image, parameters)
    {
        ## parameters should be optional, if absent use the default ones by not uploading anything
        if(missing(parameters)) {
            body_list <- list(images_file=upload_file(image))
        } else {
            body_list <- list(parameters=upload_file(parameters), images_file=upload_file(image))
        }
        POST(sprintf("%s/v3/classify?api_key=%s&version=2016-05-20", api_endp, api_key), body = body_list, encode = "multipart")
    }

We can start with a single random sample just to see how it works.

In [ ]:
## Pick a random sample from the training set
sample <- testing_set[sample.int(nrow(testing_set),1), ]

Let's remember the structure of our data frame through this single random observation:

In [ ]:
str(sample)
head(testing_set)

Using our `classify_image` function we ask Watson to classify our image:

In [ ]:
classification  <- classify_image(api_key, api_endp, sample$File)

## Display the classification result
result <- content(classification, "text")
result

## Read the JPEG
jpeg <- readJPEG(sprintf("PH2Dataset/PH2 Dataset images/%s/%s_Dermoscopic_Image/%s.jpg", sample$Name, sample$Name, sample$Name))
## Display the JPEG
grid.raster(jpeg)

That's a lot of matches, why? Because we have asked Watson to classify the image based on the default set of classifiers, i.e. existing models which are built-in Watson Visual Recognition and which we can use without having to train any specific classifier; what we want though is to ask Watson to only use the classifier we trained and nothing else: for this we will submit a _parameters_ file which should contain the `classifier_id` of our model.

Let's create the file:

In [ ]:
## Create the JSON string, save it and display it
parameters <- "parameters.json"
write(toJSON(list(classifier_ids=classifier_id)), parameters)
print(readLines(parameters), quote=FALSE)

We can now ask Watson to give us the classification based solely on our classifier:

In [ ]:
## Ask for classification 
classification  <- classify_image(api_key, api_endp, parameters, sample$File)

## The request is in binary so we first obtain the text part of the content
result <- fromJSON(content(classification, "text"))

We got our first answer from Watson's Visual Recognition using our model; let's see how what it is:

In [ ]:
print(result)

The result will vary: if it didn't match then there will be no indication of "melanoma" in the output, meaning that for Watson the image is of a benign lesion. We can parse the result to produce a more understandable output.

In [ ]:
if (class(result$images$classifiers[[1]][1]) == "data.frame")
    {
        ## Only a positive match returns a data frame
        sample$Watson = 2
        sample$Score = result$images$classifiers[[1]]$classes[[1]]$score
        
    } else {
        
        ## A list is returned since there was no match
        sample$Watson = 0
        sample$Score = "NA"
        
    }

 sprintf("Sample image was %s, Watson identified it as %s (score %s)", sample$Clinical.Diagnosis, sample$Watson, sample$Score)

Depending on the specific sample we can get a match, a false positive or a false negative (type I and type II error), which will influence _specificity_ and _sensibility_ as we will see.

## Classify all the test cases

To see how accurate is our mode we must now classify all images and store the result so we can latter analyse the results. We will want to store the reply Watson provides along with the score so we'll add two new columns to the testing set. To make the process as clear as possible I've added more `print` statements than usual

In [ ]:
## Add new columns to store results
testing_set$Watson  <- ""
testing_set$Score  <- ""

sprintf("Looping to %s", nrow(testing_set))
for (i in 1:nrow(testing_set))
{
    case <- testing_set[i, ]
    classification  <- classify_image(api_key, api_endp, parameters, case$File)
    result <- fromJSON(content(classification, "text"))
    print(result)

    ## Depending on the reply we have a match or not
    if (class(result$images$classifiers[[1]][1]) == "data.frame")
        {
            ## Only a positive match returns a data frame
            testing_set[i, ]$Watson = 2
            testing_set[i, ]$Score = result$images$classifiers[[1]]$classes[[1]]$score
            
        } else {

            ## A list is returned since there was no match
            testing_set[i, ]$Watson = 0
            
        }
    
    print(sprintf("Sample image was %s, Watson identified it as %s (score %s)", testing_set[i, ]$Clinical.Diagnosis, testing_set[i, ]$Watson, testing_set[i, ]$Score))

}

## Analysing the results

We now have all the data in our data frame and can make some quick analysis right away by comparing the `Clinical.Diagnosis` field with the `Watson` one:

In [ ]:
testing_set[, c("Name","Clinical.Diagnosis","Watson","Score") ]

Since we are using `R` we have at our disposable the necessary tools to make a more in-depth analysis of the results, starting from a simple contingency table:

In [ ]:
## A simple table
table(testing_set$Clinical.Diagnosis, testing_set$Watson)

The `caret` library contains a useful function: `confusionMatrix`, exactely what we need to get information on the accuracy, specificity and sensibility of our model:

In [ ]:
## Confusion Matrix
confusionMatrix(table(testing_set$Clinical.Diagnosis, testing_set$Watson))

A more visual analysis is provided by a four-fold plot:

In [ ]:
## Four fold plot
fourfoldplot(table(testing_set$Clinical.Diagnosis, testing_set$Watson), conf.level = 0,margin=1, color = c("#CC6666", "#99CC99"))

The results will vary at each run since we are randomly building our training and testing set; in general the model display a very high specificity (with little or no false positives) and a sensibility that varies from very high to high (i.e. some false negatives can appear in some runs).

# Conclusion

In this workshop we went from getting a zip-file with images to having an image analysis model based on Watson's Visual Recogntion, a "deep learning" convolutional neural network that is easily available through Bluemix. We have done a lot of work but barely scratched the surface of what can be done, especially when considering that there are many other Watson APIs which can be used.

Some challenges that could very well be the basis for further workshops and experiments:

* What happens if we do not remove the "atypical nevus" entries from the training dataset?
* How does the current model classify those atypical lesions (hint: it's just a matter of reruning the classifier with a different set)
* Submitting images via a web interface for classificaiton (hint: Node RED is available on Bluemix and allows for a visual construction of flows with direct support for Watson APIs
* Applying the masks in the original database increases accuracy? (hint: this would need image conversion and saving the results from several runs; Watson Analytics could be used to analyse the final results)
* Add a natural language interface that talks to a user, asks for images, uploads them and provides the result (hint: multiple Watson APIs can be used here, from those around speech and text to Watson Conversation).

These are but some initial ideas, many more are possible and hopefully this small experiment with Visual Recognition will be a starting point.

# Bibliography

```
Bhuiyan, M. A. H., Azad, I., & Uddin, M. K. (2013). Image processing for skin cancer features extraction. International Journal of Scientific & Engineering Research, 4(2), 1–6.
Carli, P., Mannone, F., de Giorgi, V., Nardini, P., Chiarugi, A., & Giannotti, B. (2003). The problem of false-positive diagnosis in melanoma screening: the impact of dermoscopy. Melanoma Research, 13(2), 179–182.
Coll, L. R., Chinchilla, D., Coll, C., Stengel, F. M., & Cabo, H. (2010). Digital image analysis of pigmented skin lesions. Early diagnosis of melanoma. Dermatología Argentina, 14(SE), 2008; 14 (3): 200-206.
Dicker, D. T., Lerner, J., Van Belle, P., Guerry, 4th, Herlyn, M., Elder, D. E., & El-Deiry, W. S. (2006). Differentiation of normal skin and melanoma using high resolution hyperspectral imaging. Cancer Biology & Therapy, 5(8), 1033–1038.
Gutman, D., Codella, N. C., Celebi, E., Helba, B., Marchetti, M., Mishra, N., & Halpern, A. (2016). Skin Lesion Analysis toward Melanoma Detection: A Challenge at the International Symposium on Biomedical Imaging (ISBI) 2016, hosted by the International Skin Imaging Collaboration (ISIC). arXiv Preprint arXiv:1605.01397.
Linden, A. (2006). Measuring diagnostic and predictive accuracy in disease management: an introduction to receiver operating characteristic (ROC) analysis. Journal of Evaluation in Clinical Practice, 12(2), 132–139.
Mendonça, T., Ferreira, P. M., Marques, J. S., Marcal, A. R., & Rozeira, J. (2013). PH 2-A dermoscopic image database for research and benchmarking. In Engineering in Medicine and Biology Society (EMBC), 2013 35th Annual International Conference of the IEEE (pp. 5437–5440). IEEE.
Menegola, A., Fornaciali, M., Pires, R., Bittencourt, F. V., Avila, S., & Valle, E. (2017). Knowledge Transfer for Melanoma Screening with Deep Learning. arXiv Preprint arXiv:1703.07479.
Nylund, A. (2016). To be, or not to be Melanoma: Convolutional neural networks in skin lesion classification.
```
